<a href="https://colab.research.google.com/github/pejmanrasti/Computer-Vision/blob/main/Jour_3/TD_ML_5_2_Spotcheck_6_AjustementFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Définir le problème


2. Récupérer les données


3. Analyser et nettoyer les données
  

4. Préparer les données
  

5. **Evaluer plusieurs modèles**
  

6. **Réglage fin des modèles**


7. Surveiller son modèle

# 📌 5. Evaluer plusieurs modèles : validation croisée

In [ ]:
# Import des bibliothèques nécessaires
import warnings  # Pour éviter les warnings inutiles

import pandas as pd  # Pour la manipulation et l'analyse des données structurées
import numpy as np  # Pour les opérations numériques et le traitement des tableaux
import matplotlib.pyplot as plt  # Pour la création de graphiques et de visualisations
import seaborn as sns  # Pour des visualisations statistiques avancées basées sur Matplotlib

# Bibliothèques Scikit-learn pour la préparation des données et le ML
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV
)  # Pour diviser les données, validation croisée et recherche de grille
from sklearn.preprocessing import StandardScaler  # Pour la normalisation des données
from sklearn.feature_selection import SelectKBest, mutual_info_classif  # Pour la sélection de variables
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
)  # Classifieurs basés sur des ensembles d'arbres
from sklearn.linear_model import LogisticRegression  # Pour la régression logistique
from sklearn.svm import SVC  # Pour les machines à vecteurs de support
from sklearn.neighbors import KNeighborsClassifier  # Pour le classifieur des k plus proches voisins
from sklearn.naive_bayes import GaussianNB  # Pour le classifieur Naive Bayes Gaussien
from sklearn.neural_network import MLPClassifier  # Pour les réseaux de neurones
from sklearn.pipeline import Pipeline  # Pour créer des pipelines de traitement
from sklearn.metrics import (
    roc_auc_score, roc_curve, RocCurveDisplay, auc, precision_recall_curve,
    average_precision_score, precision_score, recall_score, make_scorer,
    accuracy_score, confusion_matrix
)  # Pour l'évaluation des performances du modèle
from imblearn.under_sampling import RandomUnderSampler  # Pour le sous-échantillonnage des données déséquilibrées

from sklearn.datasets import make_classification  # Pour générer des jeux de données de classification
from sklearn.inspection import permutation_importance  # Pour l'importance des variables par permutation

import ipywidgets as widgets  # Pour créer des widgets interactifs dans Jupyter
from IPython.display import display, clear_output  # Pour afficher et effacer les sorties dans Jupyter

warnings.filterwarnings("ignore")  # Désactiver les warnings pour ne pas surcharger l'affichage

## 🔹5.1. Chargement des données

<font size = 4> ➡️ **Séparation des données en entraînement et test** </font>


Nous séparons le dataset en :
- **70% d'entraînement** pour apprendre au modèle.
- **30% de test** pour évaluer ses performances sur des données jamais vues.

<font size = 4> ➡️ **Sous-échantillonnage de la classe majoritaire** </font>


Nous équilibrons la base pour éviter un déséquilibre qui pourrait biaiser l'entraînement.

<font size = 4> ➡️ **Normalisation des variables** </font>


Les modèles sensibles aux échelles de variables (**SVM, MLP, Régression Logistique**) nécessitent une normalisation.

In [ ]:
# Chargement des données
df = pd.read_csv("/content/DataSet_RegionPelvienne_Class.csv", sep=",")
df = df.drop(columns=["Unnamed: 0"])  # Suppression de colonnes inutiles

# Aperçu des données
display(df.head())

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)  # Ignore les FutureWarning

# Séparation des variables explicatives et de la cible
y = df['Echec']
X = df.drop(columns=['Echec'])

# Division des données en jeu d'entraînement (70%) et de test (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=2025)

# Rééquilibrage des classes dans l'entraînement
# On fixe un ratio de 1/3 pour la classe positive et 2/3 pour la classe négative
rus = RandomUnderSampler(sampling_strategy={0: len(y_train[y_train == 1]) * 2, 1: len(y_train[y_train == 1])}, random_state=2025)
X_train_res, y_train_res = rus.fit_resample(X_train, y_train)

# Standardisation des variables explicatives
scaler = StandardScaler()
X_train_res_std = scaler.fit_transform(X_train_res)
X_test_std = scaler.transform(X_test)

# Conversion en DataFrame pour conserver les noms des colonnes
X_train_res_std = pd.DataFrame(X_train_res_std, columns=X_train_res.columns, index=X_train_res.index)
X_test_std = pd.DataFrame(X_test_std, columns=X_test.columns, index=X_test.index)


## 🔹5.2 Spotcheck

<font size = 4>  🔍 **Objectif du Spotcheck**   </font>


Avant d'optimiser finement un modèle de Machine Learning, on peut **tester rapidement plusieurs algorithmes** pour identifier ceux qui ont le plus de potentiel sur un jeu de données donné. Cette approche permet de **gagner du temps** et d'éviter d'optimiser un modèle qui ne conviendrait pas aux données.

<font size = 4>🎯 **Avantages du Spotcheck** </font>  
1. **Évaluation rapide des performances**  
   - Permet de comparer plusieurs modèles sans réglages complexes.  
   - Utilisation d'une **validation croisée** pour obtenir des résultats robustes.  

2. **Sélection des modèles les plus prometteurs**  
   - Certains algorithmes peuvent être inadaptés à la structure des données.  
   - On garde uniquement les modèles offrant les **meilleures performances initiales**.  

3. **Économie de ressources**  
   - Éviter une recherche exhaustive d’hyper-paramètres sur des modèles peu performants.  
   - Réduction du **temps de calcul** en ciblant l’optimisation sur un sous-ensemble restreint de modèles.  

<font size = 4>⚠️ **Limites du Spotcheck**  </font>  
- Les performances obtenues ne sont **pas définitives** car les modèles utilisent leurs paramètres par défaut.  
- Un modèle qui semble peu performant à ce stade pourrait s'améliorer avec une **optimisation poussée**.  





<img src="https://www.sharpsightlabs.com/wp-content/uploads/2024/02/5-fold-cross-validation_SIMPLE-EXAMPLE_v2.png" alt="drawing" width="700"/>

In [ ]:
# Définition des modèles à tester
models_base = {
    "Logistic Regression": LogisticRegression(max_iter=10000, random_state=2025),
    "Random Forest": RandomForestClassifier(random_state=2025),
    "Gradient Boosting": GradientBoostingClassifier(random_state=2025),
    "AdaBoost": AdaBoostClassifier(random_state=2025),
    "SVM": SVC(kernel="linear", probability=True, random_state=2025),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "MLP Classifier": MLPClassifier(learning_rate = 'adaptive', solver = 'adam', max_iter=10000, random_state=2025)
}

# Définition de la validation croisée : 5 folds stratifiés
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2025)


In [ ]:
%%time

# Fixer la graine aléatoire pour garantir la reproductibilité
np.random.seed(2025)

# Stockage des scores
results = {}
plt.figure(figsize=(12, 8))

# Évaluation de chaque modèle avec cross-validation
for name, model in models_base.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Normalisation des données
        ('feature_selection', SelectKBest(score_func=mutual_info_classif, k=30)),  # Sélection des variables
        ('classifier', model)  # Modèle de classification
    ])

    # Cross-validation en utilisant l'AUC-ROC comme métrique
    scores = cross_val_score(pipeline, X_train_res, y_train_res, cv=cv, scoring='roc_auc', n_jobs=-1)
    results[name] = scores.mean()  # Moyenne des scores obtenus

    # Prédictions pour le calcul de la courbe ROC (cross_val_predict avec la méthode predict_proba retourne les probabilités prédite pour chaque fold de validation de la CV)
    y_scores = cross_val_predict(pipeline, X_train_res, y_train_res, cv=cv, method="predict_proba")[:, 1]
    fpr, tpr, _ = roc_curve(y_train_res, y_scores)

    # Affichage de la courbe ROC pour chaque modèle
    plt.plot(fpr, tpr, label=f"{name} (AUC = {scores.mean():.2f})")

# Ajout d'une ligne diagonale pour le classifieur aléatoire
plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label="Classifieur Aléatoire")

# Mise en forme de la courbe ROC
plt.xlabel("Taux de Faux Positifs (FPR)")
plt.ylabel("Taux de Vrais Positifs (TPR)")
plt.title("Courbes ROC des Modèles")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


In [ ]:
# Affichage des résultats sous forme de DataFrame
results_df = pd.DataFrame.from_dict(results, orient='index', columns=["AUC-ROC"])

# Arrondi des valeurs à 2 chiffres significatifs
results_df["AUC-ROC"] = results_df["AUC-ROC"].apply(lambda x: f"{x:.2f}")

# Tri des résultats par ordre décroissant d'AUC-ROC
results_df = results_df.sort_values(by="AUC-ROC", ascending=False)

# Affichage des résultats
print("\n📊 Résultats du Spotcheck (AUC-ROC moyen sur CV) :")
print(results_df)



## 🔹 5.3. Hyperparamètres et optimisation
<font size = 4> ➡️ <span style="color:#3498db ">**Qu'est-ce qu'un hyper-paramètre ?** </span></font>


Un **hyper-paramètre** est une valeur définie **avant l'entraînement d'un modèle** et qui contrôle le processus d'apprentissage. Contrairement aux **paramètres internes** (comme les poids dans une régression ou les nœuds d’un arbre de décision), **les hyper-paramètres ne sont pas appris automatiquement** par l’algorithme, mais doivent être choisis par l'utilisateur.





<font size = 4> ➡️ <span style="color:#3498db ">**Différence entre hyper-paramètres et paramètres internes** </span>  </font>


| Type | Définition | Exemples |
|------|------------|----------|
| **Hyper-paramètres** | Choix définis avant l'entraînement qui influencent la manière dont le modèle apprend | - `C` dans une régression logistique (force de la régularisation) <br> - `n_estimators` dans une Forêt Aléatoire (nombre d'arbres) <br> - `learning_rate` dans un Gradient Boosting (taux d'apprentissage) |
| **Paramètres internes** | Valeurs apprises automatiquement à partir des données | - Les coefficients (`coef_`) dans une régression logistique <br> - Les seuils de décision d'un arbre <br> - Les poids des neurones dans un réseau de neurones |

<font size = 4> ➡️ <span style="color:#3498db ">**Exemples d'hyper-paramètres par algorithme**   </span></font>


- **Régression Logistique** : `C` (contrôle la régularisation), `penalty` (choix entre L1/L2)
- **SVM (Support Vector Machine)** : `kernel` (linéaire, RBF...), `C` (régularisation), `gamma` (influence d’un point)
- **Forêt Aléatoire** : `n_estimators` (nombre d'arbres), `max_depth` (profondeur max des arbres)
- **Réseaux de Neurones** : `hidden_layer_sizes` (nombre de neurones cachés), `learning_rate` (taux d’apprentissage)

<font size = 4> ➡️ <span style="color:#3498db ">**Pourquoi les hyper-paramètres sont-ils importants ?**  </span> </font>


Le choix des **hyper-paramètres influence directement la performance du modèle** :
- **Trop de complexité** → risque de **sur-apprentissage** (overfitting) = bon sur l'entraînement, mauvais sur les nouvelles données.
- **Pas assez de complexité** → risque de **sous-apprentissage** (underfitting) = le modèle ne capture pas assez bien les tendances.

<font size = 4> ➡️ <span style="color:#3498db ">**Comment optimiser les hyper-paramètres ?**   </span></font>


Les hyper-paramètres doivent être **optimisés** pour maximiser les performances du modèle. Plusieurs techniques existent :
- **Recherche sur grille (`GridSearchCV`)** : teste toutes les combinaisons possibles dans un espace défini.
- **Recherche aléatoire (`RandomizedSearchCV`)** : teste un nombre limité de combinaisons sélectionnées aléatoirement.
- **Optimisation bayésienne** : trouve progressivement les meilleures valeurs en fonction des résultats précédents (avec `BayesSearchCV` issue de la librairie `Scikit-Optimize`).
- **Succesive Halving (`HalvingGridSearchCV` et `HalvingRandomSearchCV`)** : teste les combinaisons d’hyperparamètres avec une allocation progressive des ressources (données et itérations).

Les hyper-paramètres sont optimisés via une CV en se basant sur un **score**. La plupart du temps on peut utiliser ceux définis dans la bibliothèque `metrics` de Scikit comme 'roc_auc', 'recall', 'precision' 🔗 [Documentation Scikit - Metriques]( https://scikit-learn.org/stable/modules/model_evaluation.html#string-name-scorers)     
On peut aussi créer sa **propre métrique** de performance qui sera utilisée pendant la CV.    
Pour cela il faut respecter les règles éditées par Sklearn : 🔗 [Documentation Scikit - MetriquePerso](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-callable)   

Exemple :
````python
def AUPRC(clf, X, y_true):   
    y_pred_proba = clf.predict_proba(X)   
    precision, recall, _ = precision_recall_curve(y_true, y_pred_proba[:, 1])   
    auc_score = metrics.auc(recall, precision)   
    return (auc_score)
````

<img src="https://www.researchgate.net/publication/364680393/figure/fig2/AS:11431281092692424@1666930838216/Tuning-of-hyperparameters-using-fivefold-cross-validation-GridSearchCV.png" alt="drawing" width="800"/>

<img src="https://miro.medium.com/v2/resize:fit:1400/1*Xq9OvMKXhrF3W2RBJCWW7w.png" alt="drawing" width="600"/>

<font size = 4> ➡️ **On sélectionne les 5 algorithmes les plus prometteurs :** </font>
- Random forest
- Gradient Boosting
- SVM
- Logistic Regression
- MLP Classifier

<font size = 4> <span style="color:#2980b9">🌳 **Impact des hyper-paramètres sur les performances : Exemple avec Random Forest** </span></font>

<img src="https://miro.medium.com/v2/resize:fit:1400/1*b3kM3WFJB6b92ZHp_J871Q.jpeg" alt="drawing" width="600"/>

Une **forêt aléatoire** (Random Forest) est un **algorithme d'apprentissage supervisé** basé sur un ensemble d'arbres de décision. Il fonctionne selon le principe suivant :

1. **Création de plusieurs arbres de décision** à partir d'échantillons aléatoires des données d'entraînement (**bootstrap sampling**).
2. **Prédiction par vote majoritaire** (pour la classification) ou **moyenne des prédictions** (pour la régression).


| Hyper-paramètre | Description |
|---------------|------------|
| `n_estimators` | Nombre d'arbres dans la forêt |
| `max_depth` | Profondeur maximale des arbres |
| `min_samples_split` | Nombre minimum d’échantillons pour diviser un nœud |
| `min_samples_leaf` | Nombre minimum d’échantillons par feuille |
| `max_features` | Nombre de variables testées à chaque division |
| `bootstrap` | Utilisation d’échantillons aléatoires avec remise pour construire chaque arbre |
| `class_weight` | Poids des classes (utile pour données déséquilibrées) |



<font size = 3>➡️ **On teste l'impact de 3 hyper-paramètres, pris indépendamment, sur les performances de la forêt aléatoire mesurées avec l'AUROC :** </font>

In [ ]:
%%time
# Fixer la graine aléatoire pour garantir la reproductibilité
np.random.seed(2025)

# Définition du modèle de base
rf = RandomForestClassifier(random_state=2025, n_jobs=-1)

# Définition de la validation croisée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2025)

# Définition des hyper-paramètres à tester
param_grids = {
    "n_estimators": [3, 5, 7, 10, 12, 14, 16, 20, 25, 30, 35, 40, 45,  50, 70, 100],  # Nombre d'arbres
    "max_depth": [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20],  # Profondeur max
    "min_samples_split": [2, 3, 4, 5,6, 7, 8, 9, 10, 15, 17, 20]  # Nombre minimum d'échantillons pour diviser un nœud
}

# Stockage des résultats
results = {}

# Recherche sur grille pour chaque hyper-paramètre
for param, values in param_grids.items():
    print(f"Optimisation de {param}...")

    grid = GridSearchCV(
        rf,
        param_grid={param: values},
        cv=cv,
        scoring="roc_auc",
        n_jobs=-1,
        return_train_score=False
    )

    # Entraînement sur l'ensemble des données d'entraînement
    grid.fit(X_train_res_std, y_train_res)

    # Stockage des résultats
    results[param] = {
        "values": values,
        "mean_score": grid.cv_results_["mean_test_score"],
        "std_score": grid.cv_results_["std_test_score"]
    }

# Tracé des courbes AUROC pour chaque hyper-paramètre
plt.figure(figsize=(15, 5))

for i, (param, res) in enumerate(results.items()):
    plt.subplot(1, 3, i + 1)
    plt.plot(res["values"], res["mean_score"], 'o-', label="AUROC moyen")
    plt.fill_between(
        res["values"],
        np.array(res["mean_score"]) - np.array(res["std_score"]),
        np.array(res["mean_score"]) + np.array(res["std_score"]),
        alpha=0.2, label="Écart-type"
    )
    plt.xlabel(param)
    plt.ylabel("AUROC")
    plt.ylim(0.7, 0.95)
    plt.title(f"Effet de {param} sur AUROC")
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()


<font size = 4> <span style="color:#2980b9"> 📈 **Impact de la métrique de performance sur le choix des hyper-paramètres : Exemple avec la régression logistique** </span> </font>

<img src="https://datatab.fr/assets/tutorial/Logistic-function.png" alt="drawing" width="400"/>

La **régression logistique** comporte trois principaux hyper-paramètres : le type de régularisation `penalty`, la force de la régularisation `C` et le `solver` (l'algorithme utilisé pour minimiser la fonction coût - qui quantifie l'erreur entre les prédictions et les valeurs réelles -  en ajustant les poids du modèle de manière itérative).   
> La **régularisation** permet de limiter le **sur-apprentissage** (overfitting) en limitant la complexité du modèle. En effet, plus un modèle est complexe plus il risque d'avoir appris *par coeur* les données d'entrainement au lieu d'avoir appris *à partir* des données d'entrainement et donc il risque d'être mauvais sur de nouvelles données (mauvaise généralisation). Un terme de régularisation est donc ajouté à la fonction coût du modèle (dont le type est souvent désigné par l'hyper-paramètre "Penalty" et la force par l'hyper-paramètre "C") afin de réduire ses degrés de liberté (par exemple imposer des contraintes sur le poids des coefficients pour un modèle linéaire).    
Les différents types de régularisation sont :
>- l1 (ou Lasso pour Least Absolute Shrinkage and Selection Operator) :  diminue le nombre de variables du modèle
>- l2 (ou Ridge) : limite le poids des variables
>- ElasticNet : combinaison de l1 et l2.

In [ ]:

# Définition du modèle
lr = LogisticRegression(max_iter=1000, random_state=2025)

# Définition de la validation croisée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2025)

# Définition des valeurs de C à tester
param_grid = {"C": np.arange(0.01,1,0.05)}

# Définition des métriques utilisées
scoring_metrics = {
    "AUROC": make_scorer(roc_auc_score),
    "Accuracy": make_scorer(accuracy_score),
}

# Recherche des hyperparamètres via GridSearchCV
grid = GridSearchCV(
    lr,
    param_grid=param_grid,
    cv=cv,
    scoring=scoring_metrics,
    n_jobs=-1,
    return_train_score=False,
    refit="AUROC"  # Optimisation par défaut sur AUROC
)

# Entraînement sur l'ensemble des données d'entraînement
grid.fit(X_train_res_std, y_train_res)

# Stockage des résultats
results = {
    "values": param_grid["C"],
    "mean_AUROC": grid.cv_results_["mean_test_AUROC"],
    "std_AUROC": grid.cv_results_["std_test_AUROC"],
    "mean_Accuracy": grid.cv_results_["mean_test_Accuracy"],
    "std_Accuracy": grid.cv_results_["std_test_Accuracy"],
}

# Tracé des courbes pour l'hyperparamètre C
plt.figure(figsize=(8, 6))

# Affichage de l'AUROC
plt.plot(results["values"], results["mean_AUROC"], 'o-', label="AUROC", color="blue")
plt.fill_between(
    results["values"],
    np.array(results["mean_AUROC"]) - np.array(results["std_AUROC"]),
    np.array(results["mean_AUROC"]) + np.array(results["std_AUROC"]),
    alpha=0.2, color="blue"
)

# Affichage de l'Accuracy
plt.plot(results["values"], results["mean_Accuracy"], 's-', label="Accuracy", color="green")
plt.fill_between(
    results["values"],
    np.array(results["mean_Accuracy"]) - np.array(results["std_Accuracy"]),
    np.array(results["mean_Accuracy"]) + np.array(results["std_Accuracy"]),
    alpha=0.2, color="green"
)



# Trouver le meilleur C pour chaque métrique
for metric, color in zip(["AUROC", "Accuracy"], ["blue", "green"]):
    best_index = np.argmax(results[f"mean_{metric}"])
    best_value = results["values"][best_index]
    best_score = results[f"mean_{metric}"][best_index]

    # Ajout d'une ligne verticale pointillée et d'une croix pour le meilleur score
    plt.axvline(x=best_value, color=color, linestyle="--", alpha=0.7)
    plt.plot(best_value, best_score, marker="x", markersize=10, color=color, markeredgewidth=3)
    plt.annotate(f"{best_score:.2f}", (best_value, best_score + 0.005), fontsize=10, color=color)

# Paramètres d'affichage
plt.xlabel("Valeur de C (échelle logarithmique)")
#plt.xscale("log")  # On met l'axe des X en échelle logarithmique pour mieux voir l'effet de C
plt.ylabel("Score")
plt.ylim(0.5, 1)
plt.title("Impact de C sur les scores pour la Régression Logistique")
plt.legend()
plt.grid(True)

plt.show()


<div class="alert alert-block alert-info">
<b>Application </b> Afficher les performances pour des algo pour différents hyper-paramètres : </div>

In [ ]:
# Fixer la graine aléatoire pour garantir la reproductibilité
np.random.seed(2025)

# Définition des modèles disponibles
models_test = {
    "Random Forest": RandomForestClassifier(random_state=2025, n_jobs=-1),
    "Gradient Boosting": GradientBoostingClassifier(random_state=2025),
    "SVM": SVC(probability=True, random_state=2025),
    "Logistic Regression": LogisticRegression(max_iter=10000, random_state=2025),
    "MLP Classifier": MLPClassifier(max_iter=500, random_state=2025)
}

# Définition des hyper-paramètres numériques pour chaque modèle
param_options = {
    "Random Forest": {
        "n_estimators": (10, 300, 10),
        "max_depth": (2, 100, 5),
        "min_samples_split": (2, 50, 2),
        "max_features": (1, X_train_res_std.shape[1], 1),
        "min_samples_leaf": (1, 50, 1),
    },
    "Gradient Boosting": {
        "n_estimators": (10, 300, 10),
        "learning_rate": (0.01, 1.0, 0.1),
        "max_depth": (2, 100, 5),
        "min_samples_split": (2, 50, 2),
        "min_samples_leaf": (1, 50, 1),
    },
    "SVM": {
        "C": (0.01, 5, 0.1),
        "gamma": (0.01, 1, 0.1),
    },
    "Logistic Regression": {
        "C": (0.01, 1, 0.01),
    },
    "MLP Classifier": {
        "hidden_layer_sizes": (10, 200, 10),
        "alpha": (0.01, 0.1, 0.01),
        "learning_rate_init": (0.01, 1, 0.1),
        "max_iter": (100, 1000, 50),
    }
}

# Sélection du modèle
model_select = widgets.Dropdown(
    options=models_test.keys(),
    value="Random Forest",
    description="Modèle"
)

# Sélection des hyper-paramètres
param_select = widgets.SelectMultiple(
    options=list(param_options["Random Forest"].keys()),
    value=["n_estimators"],
    description="Paramètres"
)

# Mise à jour des hyper-paramètres lorsque l'utilisateur change de modèle
def update_params(*args):
    param_select.options = list(param_options[model_select.value].keys())
    param_select.value = list(param_options[model_select.value].keys())[:1]  # Sélection d'un hyper-paramètre par défaut

model_select.observe(update_params, names='value')

# Bouton pour lancer l'optimisation
button = widgets.Button(description="Lancer l'optimisation")

# Zone de sortie pour l'affichage des résultats
output = widgets.Output()

# Fonction de test des hyper-paramètres
def optimize_params(b):
    with output:
        clear_output(wait=True)  # Efface l'affichage précédent

        selected_model_name = model_select.value
        selected_model = models_test[selected_model_name]
        selected_params = {param: param_options[selected_model_name][param] for param in param_select.value}
        results = {}

        print(f"Optimisation du modèle {selected_model_name}...")

        # Recherche sur grille pour chaque hyper-paramètre sélectionné
        for param, (min_val, max_val, step) in selected_params.items():
            values = np.arange(min_val, max_val + step, step)
            print(f"Optimisation de {param} sur {len(values)} valeurs...")

            grid = GridSearchCV(
                selected_model,
                param_grid={param: values},
                cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=2025),
                scoring="roc_auc",
                n_jobs=-1,
                return_train_score=False
            )

            # Entraînement sur l'ensemble des données d'entraînement
            grid.fit(X_train_res_std, y_train_res)

            # Stockage des résultats
            results[param] = {
                "values": values,
                "mean_score": grid.cv_results_["mean_test_score"],
                "std_score": grid.cv_results_["std_test_score"]
            }

        # Tracé des courbes AUROC
        plt.figure(figsize=(15, 5))
        for i, (param, res) in enumerate(results.items()):
            plt.subplot(1, len(results), i + 1)
            plt.plot(res["values"], res["mean_score"], 'o-', label="AUROC moyen")
            plt.fill_between(
                res["values"],
                np.array(res["mean_score"]) - np.array(res["std_score"]),
                np.array(res["mean_score"]) + np.array(res["std_score"]),
                alpha=0.2, label="Écart-type"
            )
            plt.xlabel(param)
            plt.ylabel("AUROC")
            plt.ylim(0.6, 1)
            plt.title(f"Effet de {param} sur AUROC")
            plt.legend()
            plt.grid(True)

        plt.tight_layout()
        plt.show()

# Lier la fonction au bouton
button.on_click(optimize_params)

# Affichage des widgets
display(model_select, param_select, button, output)


# 📌 6. Ajustement des modèles les plus prometteurs

<font size = 3> **Impact combiné des hyper-paramètres sur les performances d’un modèle**  </font>


Lorsqu'on ajuste un modèle, chaque hyper-paramètre influence différemment la manière dont le modèle apprend à partir des données. Cependant, ce n’est pas seulement l’effet individuel de chaque hyper-paramètre qui importe, mais la combinaison de plusieurs hyper-paramètres.

<font size = 3>**Pourquoi la combinaison des hyper-paramètres est-elle importante ?**  </font>


✅ **Interactions complexes** : Un hyper-paramètre peut bien fonctionner avec une certaine valeur d’un autre hyper-paramètre, mais pas indépendamment.   
✅ **Effet de compensation** : Un hyper-paramètre peut réduire l'impact négatif d'un autre.   
✅ **Équilibre entre biais et variance** : Certains hyper-paramètres affectent le biais du modèle (ex: régularisation trop forte), d’autres la variance (ex: nombre d’arbres en random forest).   

<font size = 5>💡</font> <font size = 3>C’est pourquoi il faut faire une **recherche combinée** des hyper-paramètres via une cross-validation, plutôt que de les optimiser un par un.</font>

##  🔹 6.1 Définition des modèles et des grilles de recherche

On utilise pour cela un dictionnaire python

<img src="https://dotnettrickscloud.blob.core.windows.net/img/python/2820230312012106.webp" alt="drawing" width="300"/>

In [ ]:
# Définition des modèles avec leurs grilles d'hyper-paramètres
# On va utiliser un pipeline par la suite
# En préfixant les hyperparamètres par clf__, on indique à GridSearchCV que ces hyperparamètres appartiennent au classifieur et non à d’autres étapes du pipeline.
# Evidemment il faut nommer le classifieur "clf" lorsqu'on crée le pipeline.
models = {
    "Random Forest": {
        "model": RandomForestClassifier(criterion = 'gini', max_features = 'sqrt', class_weight = "balanced", random_state=2025),
        "params": {
            "clf__n_estimators": [100, 120, 150],
            "clf__max_depth": [6, 8, 10, 12],
        }
    },

    "MLP Classifier": {
        "model": MLPClassifier(max_iter=10000, learning_rate = 'adaptive', solver = 'adam', random_state=2025),
        "params": {
            "clf__hidden_layer_sizes": [(10,), (20,), (40,), (100,)],
            "clf__alpha": [0.0001, 0.001, 0.01, 0.1],
            "clf__activation":  ['tanh', 'relu']
        }
    },

    "SVM": {
        "model": SVC(class_weight = "balanced", probability=True, random_state=2025),
        "params": {
            "clf__C": [0.1, 0.2, 0.5],
            "clf__kernel": ["linear", "rbf"]
        }
    },

    "Logistic Regression": {
        "model": LogisticRegression(class_weight = "balanced",max_iter=10000, random_state=2025),
        "params": {
            "clf__C": [0.05, 0.1, 0.5, 0.8, 1],
            "clf__solver": ["lbfgs", "liblinear", "saga"]
        }
    },

    "Gradient Boosting": {
        "model": GradientBoostingClassifier(max_features = 'sqrt', learning_rate = 0.05, min_samples_leaf = 0.1, random_state=2025),
        "params": {
            "clf__n_estimators": [80, 100, 150],
            "clf__subsample" : [0.7, 1.0],
            "clf__min_samples_split":[2, 4, 6],
            "clf__max_depth": [3, 6, 8]

        }
    }
}


## 🔹 6.2  Recherche des meilleurs hyper-paramètres

Les paramètres de la **recherche sur grille** :
- **estimator** : le modèle à ajuster. Ici on utilise le pipeline qui intègre la normalisation des données. Il faut donc utiliser en entrée de ce pipeline les données **non** normalisées. Cela permet de rendre les folds de validation de la CV complètement indépendants à l'image de données totalement nouvelles, car les données normalisées sur les folds de validation le sont par rapport à la moyenne et l'écart-type calculés sur les folds d'entrainement grâce à la définition de ce pipeline.
- **param_grid** : les hyper-paramètres à tester
- **return_train_score** : est-ce qu'on demande à connaitre les scores obtenus sur les folds d'entrainement. Cela peut être intéressant si on veut estimer l'impact des hyper-paramètres sur la généralisation du modèle mais ça requiert également des temps de calcul plus longs.
- **cv** : les paramètres de cross-validation qui va être utilisée pour évaluer les performances du modèle. La CV est répétée autant de fois qu'il y a de combinaisons d'hyper-paramètres à tester. Un score moyen sur l'ensemble des folds de validation est obtenu pour chaque combinaison d'hyper-paramètres testée.
- **scoring** : la ou les métriques à calculer sur les folds de test. Si on veut calculer plusieurs métriques pendant la CV, il faut les définir dans un dictionnaire.
- **refit** : puisqu'on donne une liste de métriques à calculer, il faut préciser quelle métrique parmi celles fournies la fonction doit utiliser pour sélectionner la meilleure combinaison d'hyper-paramètres. C'est avec cette combinaison d'hyper-paramètres que la fonction va réentrainer le modèle final sur l'ensemble des données d'entrainements (sans fold de validation).
- **error_score** : on demande à générer une erreur et non à assigner une valeur par défaut au score si une erreur se produit lors de l'ajustement du modèle.
- **n_jobs** : nombre de processeurs à utiliser lors du calcul. -1 signifie qu'on souhaite utiliser tous les processeurs disponibles. Cela peut bloquer le PC le temps de la recherche. Si on veut garder un ou deux processeurs de libres, on peut lui donner un nombre entier qui correspond au nombre de processeurs qu'on souhaite utiliser.

In [ ]:
%%time
# Fixer la graine aléatoire pour garantir la reproductibilité
np.random.seed(2025)

# Définition de la validation croisée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2025)

# Dictionnaires pour stocker les résultats
roc_results = {}
pr_results = {}

best_models = {}
best_overall_model = None
best_overall_auroc = 0
best_overall_params = None

# Optimisation des hyperparamètres et calcul des courbes ROC et PR sur la CV
for name, info in models.items():
    print(f"🔎 Optimisation des hyperparamètres pour {name}...")

    # Définition du pipeline : standardisation + sélection des variables + modèle
    pipeline = Pipeline([
        ("scaler", StandardScaler()),  # Normalisation
        ("feature_selection", SelectKBest(score_func=mutual_info_classif, k=30)),  # Sélection des variables
        ("clf", info["model"])  # Modèle
    ])

    # Définition des métriques utilisées dans la recherche sur grille
    scoring_metrics = {"AUROC": "roc_auc", "AUPRC": "average_precision"}

    # Recherche des hyperparamètres via GridSearchCV
    grid = GridSearchCV(
        estimator = pipeline,
        param_grid=info["params"],
        return_train_score = False,
        cv=cv,
        scoring=scoring_metrics,
        n_jobs=-1,
        refit="AUROC"  # Le modèle sera réajusté avec la meilleure configuration selon l'AUROC
    )
    grid.fit(X_train_res, y_train_res)

    # Extraction du modèle avec les meilleurs hyperparamètres
    best_model = grid.best_estimator_
    best_params = grid.best_params_

    # Stockage des scores moyens de la recherche sur grille
    best_auroc = grid.cv_results_["mean_test_AUROC"][grid.best_index_]
    best_auprc = grid.cv_results_["mean_test_AUPRC"][grid.best_index_]
    best_auroc_std = grid.cv_results_["std_test_AUROC"][grid.best_index_]
    best_auprc_std = grid.cv_results_["std_test_AUPRC"][grid.best_index_]

    print(f"✔ Meilleurs hyperparamètres pour {name} : {best_params}")
    print(f"📊 Score moyen AUROC en CV : {best_auroc:.2f} +/- {best_auroc_std:.2f}")
    print(f"📊 Score moyen AUPRC en CV : {best_auprc:.2f} +/- {best_auprc_std:.2f}\n")

    # Stockage du modèle et des scores
    best_models[name] = (best_model, best_auroc, best_auprc)

    # Mise à jour du meilleur modèle global en fonction de l'AUROC
    if best_auroc > best_overall_auroc:
        best_overall_model = best_model
        best_overall_auroc = best_auroc
        best_overall_auprc = best_auprc
        best_overall_auroc_std = best_auroc_std
        best_overall_auprc_std = best_auprc_std
        best_overall_params = best_params


    pipeline_best = Pipeline([
        ("scaler", StandardScaler()),  # Normalisation
        ("feature_selection", SelectKBest(score_func=mutual_info_classif, k=30)),  # Sélection des variables
        ("clf", best_model)  # Modèle
    ])

    # Prédictions en validation croisée pour tracer les courbes ROC et PR
    y_scores = cross_val_predict(pipeline_best, X_train_res, y_train_res, cv=cv, method="predict_proba")[:, 1]

    # Calcul des courbes ROC
    fpr, tpr, _ = roc_curve(y_train_res, y_scores)
    roc_auc = auc(fpr, tpr)

    # Calcul des courbes Précision-Rappel
    precisions, recalls, _ = precision_recall_curve(y_train_res, y_scores)
    pr_auc = auc(recalls, precisions)

    # Stockage des résultats pour affichage
    roc_results[name] = (fpr, tpr, roc_auc)
    pr_results[name] = (recalls, precisions, pr_auc)

# Affichage du meilleur modèle trouvé
print(f"🏆 Meilleur modèle global : {best_overall_model.named_steps['clf'].__class__.__name__}")
print(f"🎯 Hyperparamètres optimaux : {best_overall_params}")
print(f"📈 AUROC moyen en CV : {best_overall_auroc:.2f} +/- {best_overall_auroc_std:.2f}")

# Affichage des courbes ROC pour tous les modèles
plt.figure(figsize=(10, 6))
for name, (fpr, tpr, roc_auc) in roc_results.items():
    plt.plot(fpr, tpr, label=f"{name} (AUROC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("Taux de faux positifs (1 - spécificité)")
plt.ylabel("Taux de vrais positifs (rappel)")
plt.title("Courbes ROC des modèles optimisés (Validation Croisée)")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

# Affichage des courbes Précision-Rappel pour tous les modèles
plt.figure(figsize=(10, 6))
for name, (recalls, precisions, pr_auc) in pr_results.items():
    plt.plot(recalls, precisions, label=f"{name} (AUC PR = {pr_auc:.2f})")
baseline = np.sum(y_train_res) / len(y_train_res)
plt.plot([0, 1], [baseline, baseline], linestyle='--', color='grey', label=f"Baseline AUPRC = {baseline:.2f}")
plt.xlabel("Rappel")
plt.ylabel("Précision")
plt.title("Courbes Précision-Rappel des modèles optimisés (Validation Croisée)")
plt.legend(loc="lower left")
plt.grid(True)
plt.show()


 <font size = 7>⚠️</font>

 **ne pas utiliser `cross_val_predict` pour choisir un modèle**

 Ici `cross_val_predict` a été utilisé uniquement pour illustrer le notebook avec l'affichage des courbes ROC et PR.      
 Le résultat de `cross_val_predict` peut différer de celui obtenu avec `cross_val_score` (utilisé si on n'optimise pas les hyper-paramètres) ou `GridSearchCV` (surcouche de cross_val_score où on va d'abord optimiser les hyper-paramètres), car les éléments sont regroupés différemment. La fonction `cross_val_score` (ou `GridSearchCV`) calcule un **score moyen** sur les folds de validation croisée, tandis que `cross_val_predict` renvoie les **étiquettes** (ou probabilités si on précise : method="predict_proba") prédites sur les folds de validation par plusieurs modèles distincts (car entrainés sur différents folds d'entrainement).


Lorsqu'on utilise `cross_val_predict` :   
1. **On ne capture pas la variabilité des performances** observée pendant la validation croisée.
2. **On risque un surajustement aux données d'entraînement** si on choisit le modèle basé uniquement sur `cross_val_predict` plutôt que sur les résultats de `GridSearchCV`.

Pour comparer les modèles entre eux, il faut utiliser leurs **scores moyens** obtenus pendant la **validation croisée**, et non sur `cross_val_predict`.




<font size = 7>⚠️</font>


**Valeurs par défaut** des fonctions scikit, notamment pour les modèles.

Pour les modèles ML, quand on ne précise pas la valeur de tous les paramètres des fonctions scikit, des valeurs par défaut sont utilisées (elles sont référencées dans la documentation liée à la fonction). Cependant, pour s'assurer que notre code soit le plus reproductible possible, il vaut mieux préciser ces valeurs - même si on garde les valeurs par défaut - car **les valeurs par défaut utilisées par scikit changent avec les versions de scikit**.

<div class="alert alert-block alert-info">
<b>Application </b> Impact sur les performances du modèle de la recherche des hyper-paramètres selon différentes métriques : </div>

In [ ]:


# Widget d'affichage des résultats
output = widgets.Output()

# Définition des modèles et de leurs hyperparamètres
models_test = {
    "Logistic Regression": {
        "model": LogisticRegression(max_iter=10000, random_state=2025, class_weight="balanced"),
        "params": {
            "clf__C": [0.01, 0.1, 1, 10, 100],
            "clf__solver": ["liblinear", "lbfgs", "saga"]
        }
    },
    "Random Forest": {
        "model": RandomForestClassifier(random_state=2025, class_weight="balanced"),
        "params": {
            "clf__n_estimators": [10, 50, 100, 200],
            "clf__max_depth": [5, 10, 20]
        }
    },
    "Gradient Boosting": {
        "model": GradientBoostingClassifier(random_state=2025),
        "params": {
            "clf__n_estimators": [50, 100, 200],
            "clf__learning_rate": [0.01, 0.1, 0.2]
        }
    },
    "SVM": {
        "model": SVC(probability=True, random_state=2025, class_weight="balanced"),
        "params": {
            "clf__C": [0.01, 0.1, 1, 10],
            "clf__kernel": ["linear", "rbf"]
        }
    },
    "MLP Classifier": {
        "model": MLPClassifier(max_iter=1000, random_state=2025),
        "params": {
            "clf__hidden_layer_sizes": [(50,), (100,), (50, 50)],
            "clf__alpha": [0.0001, 0.001, 0.01, 0.1]
        }
    }
}

# Liste des métriques pour l'optimisation des hyper-paramètres
scoring_options = {
    "Accuracy": "accuracy",
    "F1-score": "f1",
    "Rappel": "recall",
    "Précision": "precision",
    "AUROC": "roc_auc",
    "AUPRC": "average_precision"
}

# Widgets interactifs
model_select = widgets.SelectMultiple(
    options=list(models.keys()),
    value=["Logistic Regression", "SVM"],
    description="Modèles:"
)

scoring_select = widgets.SelectMultiple(
    options=list(scoring_options.keys()),
    value=["AUROC"],
    description="Métriques:",
    layout=widgets.Layout(width="50%", height = "100%")
)

button = widgets.Button(description="Lancer l'optimisation")

# Fonction d'optimisation et d'affichage des courbes ROC et PR sur une seule figure par modèle
def optimize_models(b):
    with output:
        clear_output(wait=True)  # Nettoie l'affichage précédent

        selected_models = model_select.value
        selected_metrics = {metric: scoring_options[metric] for metric in scoring_select.value}

        print(f"🔎 Optimisation des hyperparamètres pour {', '.join(selected_models)} en fonction de {', '.join(scoring_select.value)}...\n")

        # Dictionnaires pour stocker les résultats des courbes
        roc_results = {model: {} for model in selected_models}
        pr_results = {model: {} for model in selected_models}

        for model_name in selected_models:
            print(f"📌 Modèle en cours : {model_name}")

            model_info = models_test[model_name]

            # Définition du pipeline : standardisation + sélection des variables + modèle
            pipeline = Pipeline([
                ("scaler", StandardScaler()),  # Normalisation
                ("feature_selection", SelectKBest(score_func=mutual_info_classif, k=30)),  # Sélection des variables
                ("clf", model_info["model"])  # Modèle
            ])

            # Définition de la validation croisée
            cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2025)

            # Sélectionner une métrique au hasard pour le refit
            refit_metric = np.random.choice(list(selected_metrics.keys()))
            # Recherche des hyperparamètres via GridSearchCV avec plusieurs métriques
            grid = GridSearchCV(
                pipeline,
                param_grid=model_info["params"],
                cv=cv,
                scoring=selected_metrics,
                n_jobs=-1,
                refit=refit_metric  # On utilise une seule métrique pour le refit
            )
            grid.fit(X_train_res, y_train_res)
            results = grid.cv_results_

            # Stocker les meilleurs paramètres pour chaque métrique
            best_models = {}
            for metric_name, metric in selected_metrics.items():
                best_index = np.nonzero(results[f"rank_test_{metric_name}"] == 1)[0][0]
                best_params = results["params"][best_index]
                best_params = {key.replace("clf__", ""): value for key, value in best_params.items()}
                best_models[metric_name] = model_info["model"].set_params(**best_params)

                print(f"✔️ Meilleurs hyperparamètres pour {model_name} ({metric_name}) : {best_params}")

            # Génération des courbes ROC et PR pour chaque métrique optimisée
            for metric_name, best_model in best_models.items():
                best_pipeline = Pipeline([
                    ("scaler", StandardScaler()),
                    ("feature_selection", SelectKBest(score_func=mutual_info_classif, k=30)),
                    ("clf", best_model)
                ])

                # Prédictions en validation croisée pour la courbe ROC et PR
                y_scores = cross_val_predict(best_pipeline, X_train_res, y_train_res, cv=cv, method="predict_proba")[:, 1]

                # Calcul des courbes ROC
                fpr, tpr, _ = roc_curve(y_train_res, y_scores)
                roc_auc = auc(fpr, tpr)

                # Calcul des courbes Précision-Rappel
                precisions, recalls, _ = precision_recall_curve(y_train_res, y_scores)
                pr_auc = auc(recalls, precisions)

                print(f"📈 AUROC en CV ({metric_name}) : {roc_auc:.2f}")
                print(f"📉 AUC PR en CV ({metric_name}) : {pr_auc:.2f}\n")

                # Stockage des résultats
                roc_results[model_name][metric_name] = (fpr, tpr, roc_auc)
                pr_results[model_name][metric_name] = (recalls, precisions, pr_auc)

        # Affichage des courbes ROC et PR sur une seule figure par modèle
        for model_name in selected_models:
            fig, axes = plt.subplots(1, 2, figsize=(14, 6))

            # Courbes ROC
            for metric_name, (fpr, tpr, roc_auc) in roc_results[model_name].items():
                axes[0].plot(fpr, tpr, label=f"{metric_name} (AUROC = {roc_auc:.2f})")
            axes[0].plot([0, 1], [0, 1], linestyle="--", color="gray")
            axes[0].set_title(f"Courbes ROC - {model_name}")
            axes[0].legend(loc="lower right")

            # Courbes PR
            for metric_name, (recalls, precisions, pr_auc) in pr_results[model_name].items():
                axes[1].plot(recalls, precisions, label=f"{metric_name} (AUC PR = {pr_auc:.2f})")
            baseline = np.sum(y_train_res) / len(y_train_res)
            axes[1].plot([0, 1], [baseline, baseline], linestyle='--', color='grey', label=f"Baseline AUPRC = {baseline:.2f}")
            axes[1].set_title(f"Courbes PR - {model_name}")
            axes[1].legend(loc="lower left")

            plt.show()

# Lier la fonction au bouton
button.on_click(optimize_models)
display(model_select, scoring_select, button, output)


## 🔮 6.3 Prédictions du meilleur modèle sur les données tests

In [ ]:
# Fixer la graine aléatoire pour garantir la reproductibilité
np.random.seed(2025)

# === Prédictions sur les données test avec le meilleur modèle ===
model_final = best_overall_model.set_params(**best_overall_params).fit(X_train_res, y_train_res)
y_test_scores = model_final.predict_proba(X_test)[:, 1]


# Calcul des métriques sur les données test
fpr_test, tpr_test, _ = roc_curve(y_test, y_test_scores)
roc_auc_test = auc(fpr_test, tpr_test)

precisions_test, recalls_test, _ = precision_recall_curve(y_test, y_test_scores)
pr_auc_test = auc(recalls_test, precisions_test)

# Affichage des résultats sur les données test
print(f"\n🔬 Résultats sur les données d'entrainement :")
print(f"📊 AUROC sur train : {best_overall_auroc:.2f}")
print(f"📊 AUPRC sur train : {best_overall_auprc:.2f}")

print(f"\n🔬 Résultats sur les données test :")
print(f"📊 AUROC sur test : {roc_auc_test:.2f}")
print(f"📊 AUPRC sur test : {pr_auc_test:.2f}")

# === Affichage des courbes ROC et PR ===
plt.figure(figsize=(12, 5))

# Courbe ROC
plt.subplot(1, 2, 1)
plt.plot(fpr_test, tpr_test, label=f"AUROC = {roc_auc_test:.2f}", color="blue")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("Taux de faux positifs (1 - spécificité)")
plt.ylabel("Taux de vrais positifs (rappel)")
plt.title(f"Courbe ROC - {best_overall_model.named_steps['clf'].__class__.__name__}")
plt.legend(loc="lower right")
plt.grid(True)

# Courbe Précision-Rappel
plt.subplot(1, 2, 2)
plt.plot(recalls_test, precisions_test, label=f"AUPRC = {pr_auc_test:.2f}", color="green")
baseline = np.sum(y_test) / len(y_test)
plt.plot([0, 1], [baseline, baseline], linestyle='--', color='grey', label=f"Baseline AUPRC = {baseline:.2f}")
plt.xlabel("Rappel")
plt.ylabel("Précision")
plt.title(f"Courbe Précision-Rappel - {best_overall_model.named_steps['clf'].__class__.__name__}")
plt.legend(loc="lower left")
plt.grid(True)

plt.tight_layout()
plt.show()

<div class="alert alert-block alert-info">
<b>Application </b> Impact du seuil de décision sur le rappel et la précision : </div>

In [ ]:
# Générer les probabilités prédictives pour la classe positive sur les données test
model_final = best_overall_model.set_params(**best_overall_params).fit(X_train_res, y_train_res)
y_test_scores = model_final.predict_proba(X_test)[:, 1]

# Définition du widget interactif pour ajuster le seuil de décision
threshold_slider = widgets.FloatSlider(
    value=0.5,  # Seuil initial à 0.5
    min=0.0,
    max=1.0,
    step=0.01,
    description="Seuil",
    continuous_update=False
)

# Widget d'affichage des résultats
output = widgets.Output()

def update_threshold(threshold):
    """Met à jour la précision et le rappel en fonction du seuil sélectionné."""
    with output:
        output.clear_output(wait=True)  # Nettoie l'affichage précédent

        # Appliquer le seuil pour générer les prédictions binaires
        y_pred_thresholded = (y_test_scores >= threshold).astype(int)

        # Calcul des métriques avec gestion des erreurs
        precision = precision_score(y_test, y_pred_thresholded, zero_division=0)
        recall = recall_score(y_test, y_pred_thresholded)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_thresholded).ravel()
        specificity = tn / (tn+fp)
        # Affichage des résultats
        print("\033[1m" + f"🔹 Seuil de décision : {threshold:.2f} " + "\033[0m" + f"-> On considère que la prédiction appartient à la classe positive si sa probabilité est supérieur à {threshold*100:.0f} %")
        print(" ")
        print("\033[1m" + "cas 1️⃣ : on veut prédire les CQ qui passent pour ne pas les mesurer."+ "\033[0m")
        print(" ")
        print("➡️ Objectif : maximiser la spécificité et minimiser le taux d'échec (1-sensiblité)")
        print(f"🔸 Spécificité : {specificity:.2f} -> {specificity*100:.0f}% des arcs pass sont correctement détectés comme pass")
        print("\033[1m" + f"🔸 Taux d'échec : {1-recall:.2f} ->  {100-recall*100:.0f}%  des arcs fail sont détectés comme pass"+ "\033[0m")
        print(" ")
        print("\033[1m" +"cas 2️⃣ : on veut prédire les CQ qui ne passent pas pour réoptimiser les plans d'emblée."+ "\033[0m")
        print(" ")
        print("➡️ Objectif : maximiser la sensibilité et minimiser le taux de fausses découvertes (1-précision)")
        print(f"🔸 Rappel/sensibilité : {recall:.2f} ->  {recall*100:.0f}% des arcs fail sont correctement détectés comme fail")
        print("\033[1m" + f"🔸 Taux de fausses découvertes : {1-precision:.2f} -> {100-precision*100:.0f}% des arcs prédits fail ne le sont pas"+ "\033[0m")

        # Tracé des courbes Précision et Rappel en fonction du seuil
        thresholds = np.linspace(0, 1, 100)
        precisions = [precision_score(y_test, (y_test_scores >= t).astype(int), zero_division=0) for t in thresholds]
        recalls = [recall_score(y_test, (y_test_scores >= t).astype(int)) for t in thresholds]

        plt.figure(figsize=(8, 5))
        plt.plot(thresholds, precisions, label="Précision", color="blue")
        plt.plot(thresholds, recalls, label="Rappel", color="green")
        plt.axvline(threshold, color="red", linestyle="--", label=f"Seuil = {threshold:.2f}")
        plt.xlabel("Seuil de décision")
        plt.ylabel("Valeur")
        plt.title("Impact du seuil de décision sur la précision et le rappel")
        plt.legend()
        plt.grid(True)
        plt.show()

# Lier le widget au callback
widgets.interactive(update_threshold, threshold=threshold_slider)

# Affichage des widgets
display(threshold_slider, output)


## 🔹 6.4. Interpréter le modèle

La <font size = 3> **Permutation Importance** </font>est une technique permettant d'évaluer l'importance des variables dans un modèle de Machine Learning en mesurant l'effet de la permutation aléatoire de chaque variable sur la performance du modèle.

<font size = 4> ⚙️ **Comment ça marche ?** </font>
1. **Calcul du score initial** : On évalue la performance du modèle sur les données de test avec les valeurs originales des variables.
2. **Permutation d'une variable** : On remplace aléatoirement les valeurs d'une seule variable tout en gardant les autres inchangées.
3. **Calcul du nouveau score** : On mesure la performance du modèle avec cette variable perturbée.
4. **Impact sur la performance** :
   - Si la permutation d'une variable **réduit** la performance, cela signifie qu'elle est **importante** pour les prédictions.
   - Si l'impact est **faible ou négatif**, cela indique que la variable est **peu informative** voire nuisible.

<font size = 4> 📊 **Interprétation**</font>   
- **Valeur positive élevée** → Variable fortement contributive.
- **Valeur proche de zéro** → Variable non influente.
- **Valeur négative** → La variable ajoute du bruit et nuit à la prédiction.

<font size = 4> ✅ **Avantages**</font>   
✔️ Simple à comprendre et interpréter.  
✔️ Compatible avec n'importe quel modèle de Machine Learning.  

<font size = 4> ❗ **Limitations**</font>   
⚠️ Peut être biaisé en présence de variables fortement corrélées.  
⚠️ Coût de calcul élevé pour les grands ensembles de données.  


ℹ️ D'autres méthodes existent. Elles sont décrites dans la documentation scikit : 🔗 [Documentation Scikit - Model Inspection](https://scikit-learn.org/stable/inspection.html#inspection)

In [ ]:
# Fixer la graine aléatoire pour garantir la reproductibilité des résultats
np.random.seed(2025)

# Sélectionner les indices des variables qui ont été retenues après la sélection par information mutuelle
selected_features_idx = model_final.named_steps["feature_selection"].get_support(indices=True)

# Récupérer les noms des variables sélectionnées en utilisant leurs indices
selected_features = X_test.columns[selected_features_idx]  # Liste des noms des variables retenues

# Calcul de l'importance permutation sur les données de test
# L'importance permutation mesure l'impact de chaque variable en permutant ses valeurs et en observant
# la diminution de la performance du modèle (ici, l'AUROC). Plus la baisse de performance est grande, plus
# la variable est importante pour la prédiction.

result_test = permutation_importance(
    model_final,  # Modèle entraîné
    X_test,       # Données test
    y_test,       # Labels test
    scoring="roc_auc",  # Mesure de performance utilisée : AUROC (aire sous la courbe ROC)
    n_repeats=100,  # Nombre de permutations effectuées pour chaque variable
    random_state=2025,  # Fixation du générateur aléatoire pour reproductibilité
    n_jobs=-1  # Utilisation de tous les cœurs disponibles pour accélérer le calcul
)

# Ordonner les variables en fonction de leur importance décroissante
# Les variables avec la plus grande importance seront affichées en haut du graphique.
sorted_importances_idx = result_test.importances_mean.argsort()

# Création d'un DataFrame contenant les valeurs d'importance permutation pour chaque variable
importances_test = pd.DataFrame(
    result_test.importances[sorted_importances_idx].T,  # Transposition pour format adapté
    columns=X_test.columns  # Attribution des noms des variables
)

# Affichage des résultats sous forme de boxplot
plt.figure(figsize=(30, 30))

# Définition des propriétés de la ligne médiane du boxplot
medianprops = dict(color="black", linewidth=1)

# Création du boxplot pour les variables sélectionnées
importances_test[selected_features].plot.box(
    whis=10,  # Étendue des moustaches (10 signifie 10 fois l'écart interquartile)
    vert=False,  # Affichage horizontal des boîtes
    fontsize=12,  # Taille de la police
    patch_artist=True,  # Remplissage des boîtes avec une couleur
    boxprops=dict(facecolor='orange'),  # Couleur des boîtes
    whiskerprops=dict(color='black'),  # Couleur des moustaches
    medianprops=medianprops  # Application des propriétés définies pour la médiane
)

# Ajout d'un titre au graphique
plt.title("Permutation Importance (Test Set)", fontsize=14)

# Ajout d'une ligne verticale pour indiquer la référence à 0 (absence d'impact)
plt.axvline(x=0, color="k", linestyle="--")

# Étiquetage de l'axe X
plt.xlabel("Diminution du score AUROC", fontsize=12)

# Affichage d'une grille pour améliorer la lisibilité
plt.grid(visible=True)

# Ajustement automatique de la mise en page pour éviter le chevauchement des éléments
plt.tight_layout()
# Affichage du graphique
plt.show()


In [ ]:
# Calcul de la moyenne des importances permutation pour chaque variable
mean_importances = importances_test[selected_features].mean()

# Filtrer les variables dont la moyenne d'importance est > 0.01
selected_vars = mean_importances[mean_importances > 0.02]

# Affichage des résultats
if not selected_vars.empty:
    print("Variables avec une importance permutation moyenne > 0.02 :")
    print(selected_vars.sort_values(ascending=False))
else:
    print("Aucune variable avec une importance supérieure à 0.02")


## 🔹 6.5 Se comparer avec une méthode plus simple

🔍 On va comparer les performances du modèle sur les données test avec l'utilisation d'un **simple seuil** sur les deux variables les plus importantes pour le modèle.

In [ ]:
# Récupération des deux variables les plus importantes d'après la permutation importance
top_features = importances_test.mean().sort_values(ascending=False).head(2).index
print("Deux variables les plus importantes :", top_features.tolist())

# Prédiction des probabilités avec le modèle final
y_scores_model = model_final.predict_proba(X_test)[:, 1]

# Calcul des courbes ROC et PR pour le modèle final
fpr_model, tpr_model, _ = roc_curve(y_test, y_scores_model)
roc_auc_model = auc(fpr_model, tpr_model)
precision_model, recall_model, _ = precision_recall_curve(y_test, y_scores_model)
pr_auc_model = auc(recall_model, precision_model)

# Création des figures pour les courbes ROC et PR
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Courbe ROC
axes[0].plot(fpr_model, tpr_model, label=f"Modèle Final (AUROC = {roc_auc_model:.2f})", color="blue")

# Courbe Précision-Rappel
axes[1].plot(recall_model, precision_model, label=f"Modèle Final (AUC PR = {pr_auc_model:.2f})", color="blue")

# Boucle pour afficher les courbes ROC et PR des variables individuelles
for feature in top_features:
    X_test_feature = X_test_std[feature]  # Sélection de la variable

    # Vérification de l'orientation de la courbe ROC et inversion si nécessaire
    fpr_feature, tpr_feature, _ = roc_curve(y_test, X_test_feature)
    if auc(fpr_feature, tpr_feature) < 0.5:  # Si l'AUC est < 0.5, inverser les valeurs
        X_test_feature = -X_test_feature
        fpr_feature, tpr_feature, _ = roc_curve(y_test, X_test_feature)

    # Calcul des scores
    roc_auc_feature = auc(fpr_feature, tpr_feature)
    precision_feature, recall_feature, _ = precision_recall_curve(y_test, X_test_feature)
    pr_auc_feature = auc(recall_feature, precision_feature)

    # Ajout de la courbe ROC
    axes[0].plot(fpr_feature, tpr_feature, linestyle="dashed", label=f"{feature} (AUROC = {roc_auc_feature:.2f})")

    # Ajout de la courbe PR
    axes[1].plot(recall_feature, precision_feature, linestyle="dashed", label=f"{feature} (AUC PR = {pr_auc_feature:.2f})")

# Personnalisation des courbes ROC
axes[0].plot([0, 1], [0, 1], linestyle="--", color="gray", label="Baseline AUROC = 0.5")
axes[0].set_xlabel("Taux de Faux Positifs (FPR)")
axes[0].set_ylabel("Taux de Vrais Positifs (TPR)")
axes[0].set_title("Courbes ROC - Modèle vs Variables Individuelles")
axes[0].legend()
axes[0].grid()

# Personnalisation des courbes PR
baseline = np.sum(y_test) / len(y_test)
axes[1].plot([0, 1], [baseline, baseline], linestyle='--', color='grey', label=f"Baseline AUPRC = {baseline:.2f}")
axes[1].set_xlabel("Rappel")
axes[1].set_ylabel("Précision")
axes[1].set_title("Courbes PR - Modèle vs Variables Individuelles")
axes[1].legend()
axes[1].grid()

plt.tight_layout()
plt.show()


## 💾 Sauvegarder son modèle

Enregistrer un modèle créé avec scikit-learn est une étape importante pour pouvoir le **réutiliser** sans avoir à le réentraîner.

<font size = 4> **1. Joblib** </font>

<font size = 3> ✔ **Avantages :**</font>


- Optimisé pour les objets volumineux et les modèles scikit-learn (utilisation efficace de la compression et du multi-processing).
- Plus rapide que `pickle` pour sauvegarder et charger des modèles de grande taille.
- Permet de sérialiser des numpy arrays de manière plus efficace.

<font size = 3> ❌ **Inconvénients :**</font>


- Spécifique à Python et donc **non compatible avec Microsoft .NET** ou d'autres langages.
- Ne garantit pas la compatibilité entre différentes versions de Python ou de scikit-learn.
- Ne supporte pas bien le transfert entre systèmes d’exploitation différents.

In [ ]:
import joblib
import numpy as np
from sklearn.metrics import roc_curve, auc

# Prédiction des probabilités avec le modèle final
# Utilise le modèle final pour prédire les probabilités sur l'ensemble de test
# `predict_proba` renvoie un tableau de probabilités pour chaque classe, nous prenons la probabilité de la classe positive (1)
y_scores_model = model_final.predict_proba(X_test)[:, 1]

# Calcul des courbes ROC et PR pour le modèle final
fpr_model, tpr_model, _ = roc_curve(y_test, y_scores_model)
roc_auc_model_base = auc(fpr_model, tpr_model)
print(f"AUROC du modèle de base : {roc_auc_model_base:.2f}")

# Enregistrer le modèle avec joblib
# `joblib.dump` enregistre le modèle dans un fichier binaire
# Cela permet de sauvegarder l'état du modèle pour une utilisation ultérieure
joblib.dump(model_final, 'model_final.joblib')

# Charger le modèle avec joblib
# `joblib.load` charge le modèle à partir du fichier binaire
# Cela permet de restaurer l'état du modèle pour faire des prédictions
joblib_model = joblib.load('model_final.joblib')

# Faire des prédictions sur X_test avec le modèle chargé
# Utilise le modèle chargé pour prédire les probabilités sur l'ensemble de test
# `predict_proba` renvoie un tableau de probabilités pour chaque classe, nous prenons la probabilité de la classe positive (1)
proba_joblib = joblib_model.predict_proba(X_test)[:, 1]

# Évaluer les performances du modèle chargé
# Calcul des courbes ROC et PR pour le modèle chargé
fpr_model, tpr_model, _ = roc_curve(y_test, proba_joblib)
roc_auc_model = auc(fpr_model, tpr_model)

print(f"AUROC du modèle chargé : {roc_auc_model:.2f}")

# Comparaison directe des prédictions
# Calcul de la différence moyenne entre les probabilités prédites par le modèle original et le modèle chargé
# Cela permet de vérifier que les prédictions sont cohérentes entre les deux modèles
differences = np.abs(y_scores_model - proba_joblib)
print(f"Différence moyenne entre les prédictions : {np.mean(differences):.6f}")


<font size = 4> **2. ONNX** </font>

<font size = 3> ✔ **Avantages :**</font>


- **Indépendant du langage** : ONNX permet d'exporter des modèles qui peuvent être utilisés avec **Python, C++, Java, Microsoft .NET (via ML.NET), et bien d'autres**.
- Supporte les **frameworks de deep learning** (PyTorch, TensorFlow, scikit-learn via `skl2onnx`).
- Compatible avec **Microsoft .NET (ML.NET)**, facilitant l’intégration des modèles dans le TPS Eclipse par exemple.

<font size = 3> ❌ **Inconvénients :**</font>


- Conversion plus complexe, tous les modèles ne sont pas directement compatibles (ex : certains modèles scikit-learn nécessitent une conversion avec `skl2onnx`).
- Moins flexible que Joblib pour charger un modèle et continuer son entraînement.
- Peut nécessiter une adaptation spécifique du code pour le chargement et l'utilisation du modèle.


In [ ]:
#Pour installer onnxruntime dans colab
!pip install onnxruntime

In [ ]:
import onnxruntime as rt
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn

# Prédiction des probabilités avec le modèle scikit-learn
# Utilise le modèle final pour prédire les probabilités sur l'ensemble de test
# `predict_proba` renvoie un tableau de probabilités pour chaque classe, nous prenons la probabilité de la classe positive (1)
y_scores_model = model_final.predict_proba(X_test)[:, 1]

# Calcul de l'AUROC pour le modèle scikit-learn
roc_auc_model_base = roc_auc_score(y_test, y_scores_model)
print(f"AUROC du modèle de base : {roc_auc_model_base:.2f}")

# Conversion du modèle scikit-learn en format ONNX
# `convert_sklearn` convertit un modèle scikit-learn en un modèle ONNX
# `FloatTensorType` spécifie le type de données d'entrée attendu par le modèle ONNX
model_onnx = convert_sklearn(
    model_final,
    initial_types=[('input', FloatTensorType([None, X_test.shape[1]]))],
    target_opset=12
)

# Fonction pour sauvegarder le modèle ONNX dans un fichier
def save_model(model, filename):
    with open(filename, "wb") as f:
        f.write(model.SerializeToString())

# Sauvegarde du modèle ONNX dans un fichier
save_model(model_onnx, 'model_final.onnx')

# Conversion des données de test en tableau numpy de type float32
# ONNX Runtime attend des entrées de type float32
X_test_np = X_test.astype(np.float32).to_numpy()

# Chargement du modèle ONNX avec ONNX Runtime
sess = rt.InferenceSession("model_final.onnx")

# Récupération des noms des entrées et sorties du modèle ONNX
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
label_proba = sess.get_outputs()[1].name

# Prédiction des labels avec le modèle ONNX
# `run` exécute le modèle ONNX et renvoie les prédictions
pred_onx = sess.run([label_name], {input_name: X_test_np.astype(np.float32)})[0]

# Prédiction des probabilités avec le modèle ONNX
# `run` exécute le modèle ONNX et renvoie les probabilités
proba_onx = sess.run([label_proba], {input_name: X_test_np.astype(np.float32)})[0]

# Extraction des probabilités pour la classe positive (1)
proba_onx = [d[1] for d in proba_onx]

# Calcul de l'AUROC pour le modèle ONNX
# Comparaison des probabilités prédites par le modèle ONNX avec les vraies étiquettes
roc_auc_model_onnx = roc_auc_score(y_test, proba_onx)
print(f"AUROC du modèle ONNX : {roc_auc_model_onnx:.2f}")

# Comparaison directe des prédictions entre le modèle scikit-learn et le modèle ONNX
# Calcul de la différence moyenne entre les probabilités prédites par les deux modèles
# Cela permet de vérifier que les prédictions sont cohérentes entre les deux modèles
differences = np.abs(y_scores_model - proba_onx)
print(f"Différence moyenne entre les prédictions : {np.mean(differences):.6f}")




➡ **Si vous travaillez exclusivement en Python** et avec des modèles scikit-learn, `Joblib` est le meilleur choix.  
➡ **Si vous devez déployer un modèle dans un environnement .NET** ou tout autre langage, `ONNX` est recommandé.  


<font size = 7>⚠️</font>


Toujours s'assurer que les prédictions du modèles de base python et du modèle enregistré sont les mêmes. Des différences peuvent apparaitre, notamment avec ONNX, si on ne définit pas correctement les paramètres d'enregistrement ou d'import.